# C-Y-K algoritmus

_Sipos István_  
_2020.04.14_

Az algoritmussal  tetszőleges Chomsky féle normál alakban megadott nyelvtan és tetszőleges terminális sztring esetén eldönthető (polinomiális időben), hogy a sztring eleme-e a nyelvtan által generált nyelvnek. 

## Implementáció

### Rule osztály

Egy nyelvtani szabályt reprezentál, két tulajdonsága (_left_ és _right_) a felírt szabály két oldalát jelenti.

Pl.:

$$S \rightarrow AB$$

szabály így fogalmazható meg:
```python
Role('S', ['A', 'B'])

```

In [1]:
class Rule():
    def __init__(self, left, right):
        self.left = left
        self.right = right
        

### Grammar osztály

A konstruktor paramétere a startszimbólum. A nyevtan szabályai az _add()_ metódussal adhatók hozzá. A _resolve(right)_ metódus azoknak a szabályoknak a bal oldalával tér vissza, amelyek jobb oldala a _right_ paraméter.


In [2]:
class Grammar():
    def __init__(self, start):
        self.start = start
        self.rules = []
        
    def add(self, rule):
        self.rules.append(rule)

    def resolve(self, right):
        return {rule.left for rule in self.rules if rule.right == right}

### ParseTable osztály

A CYK algoritmusban használt alsótrianguláris mátrixot valósítja meg, a konstruktor paramétere a méret.   
Az egyes mezöket _ParseTable\[sor, oszlop\]_ címzéssel lehet elérni.

In [3]:
class ParseTable():
    def __init__(self, size):
        self.size = size
        self.table = [[set() for j in range(i)] for i in range(size, 0, -1)]

    def __getitem__(self, pos):
        i, j = pos
        return self.table[i - 1][i - j]

    def __str__(self):
        result = ""
        for x in range(1, self.size + 1):
            for y in range(1, x + 1):
                result += " " + str(self[y, self.size + y - x])
            result += "\n"
        return result

### CYK osztály
A konstruktor _grammar_ paramétere egy _Grammar_ típusó nyelvtan. A _parse(word)_ függvény visszatérési értéke a egy _bool_, amely igaz ha a _grammar_ nyelvtanban levezethetö a _word_ szó, egyébként hamis, és egy _ParseTable_, ami a levezetés során elöállt mátrixot tartalmazza.

In [4]:
class CYK():
    def __init__(self, grammar):
        self.grammar = grammar

    def parse(self, word):
        parse_table = ParseTable(len(word))
        
        self.calulate_first_level(parse_table, word)

        for level in range(2, len(word)+1):
            self.calculate_level(parse_table, level)
            
        return self.grammar.start in parse_table[1, len(word)], parse_table

    def calulate_first_level(self, parse_table, word):
        for i in range(1, len(word)+1):
            parse_table[i, i].update(self.grammar.resolve([word[i-1]]))

    def calculate_level(self, parse_table, level):
        for x in range(1, parse_table.size + 2 - level):
            self.calculate_element(parse_table, x, level + x - 1)
            
    def calculate_element(self, parse_table, i, j):
        level = j - i + 1
        for x in range(level-1):
            left = parse_table[i, i + x]
            right = parse_table[i + x + 1, j]
            for l in left:
                for r in right:
                    parse_table[i, j].update(self.grammar.resolve([l, r]))

## Példa

Definiáljuk a jegyzetben példaként megtalálható nyelvtant!

In [8]:
g = Grammar(start = 'S')

g.add(Rule('S', list('AB')))
g.add(Rule('S', list('CD')))
g.add(Rule('S', list('CB')))
g.add(Rule('S', list('SS')))
g.add(Rule('A', list('BC')))
g.add(Rule('A', list('a')))
g.add(Rule('B', list('SC')))
g.add(Rule('B', list('b')))
g.add(Rule('C', list('DD')))
g.add(Rule('C', list('b')))
g.add(Rule('D', list('BA')))

Lássuk, hogy a _'aabbaba'_ szó levezethetö-e a fenti nyelvtan szabályai szerint az _S_ startszimbólumból:

In [9]:
word = 'aabbaba'
parseable, parse_table = CYK(grammar = g).parse(list(word))

In [10]:
if parseable:
    print(f'A "{word}" szó a fenti nyelvtan szabályai szerint az {g.start} startszimbólumból levezethetö, és a levezetés során a következö mátrix állt elö:')
else:
    print(f'A "{word}" szó a fenti nyelvtan szabályai szerint az {g.start} startszimbólumból nem vezethetö le. A számítás során elöállt mátrix:')

print(parse_table)


A "aabbaba" szó a fenti nyelvtan szabályai szerint az S startszimbólumból levezethetö, és a levezetés során a következö mátrix állt elö:
 {'S'}
 {'S'} {'C', 'B'}
 set() {'S'} {'A', 'D'}
 {'S'} {'D'} {'B', 'S'} {'C'}
 set() {'B'} {'S'} set() set()
 set() {'S'} {'A', 'S'} {'D'} {'S'} {'D'}
 {'A'} {'A'} {'C', 'B'} {'C', 'B'} {'A'} {'C', 'B'} {'A'}

